In [1]:
from aocd import get_data
import numpy as np

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation `X|Y` means that if both page number `X` and page number `Y` are to be produced as part of an update, page number `X` must be printed at some point before page number `Y`.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

The first section specifies the page ordering rules, one per line. The first rule, `47|53`, means that if an update includes both page number `47` and page number `53`, then page number `47` must be printed at some point before page number `53`. (`47` doesn't necessarily need to be immediately before `53`; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, `75,47,61,53,29`, means that the update consists of page numbers `75`, `47`, `61`, `53`, and `29`.

Start by identifying which updates are already in the right order.

In the above example, the first update `(75,47,61,53,29)` is in the right order:

`75` is correctly first because there are rules that put each other page after it: `75|47`, `75|61`, `75|53`, and `75|29`.

`47` is correctly second because `75` must be before it `(75|47)` and every other page must be after it according to `47|61`, `47|53`, and `47|29`.

`61` is correctly in the middle because `75` and `47` are before it (`75|61` and `47|61`) and `53` and `29` are after it (`61|53` and `61|29`).

`53` is correctly fourth because it is before page number `29` (`53|29`).

`29` is the only page left and so is correctly last.

Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

In [18]:
rules_str, updates = get_data(day=5, year=2024).split('\n\n')[:2]

In [3]:
rules = np.genfromtxt(rules_str.strip().splitlines(), delimiter="|", dtype=int)

In [26]:
rules

array([[76, 18],
       [58, 19],
       [58, 49],
       ...,
       [29, 56],
       [29, 17],
       [19, 93]])

In [4]:
from collections import defaultdict

def build_graph(rules):
    """
    Graph where the key represents the number that must come 
    ahead of any of the elements in the set of value numbers
    """
    graph = defaultdict(set)
    for x, y in rules:
        graph[x].add(y)
    return graph

rules_graph = build_graph(rules)

In [5]:
updates_1 = updates.split("\n")[0]; updates_1

'21,84,35,92,58,33,29,79,56,24,95,28,19,46,37'

In [6]:
updates_1 = np.genfromtxt(updates_1.strip().splitlines(), delimiter=",", dtype=int)

In [7]:
updates_1

array([21, 84, 35, 92, 58, 33, 29, 79, 56, 24, 95, 28, 19, 46, 37])

Work out which rules to apply:

In [8]:
matches_left = np.isin(rules[:, 0], list(updates_1))
matches_right = np.isin(rules[:, 1], list(updates_1))

both_matches = matches_left & matches_right

In [9]:
both_matches

array([False,  True, False, ...,  True, False, False])

The below set represents all the keys to look up in the rules graph to assess whether the rule is followed

In [58]:
def is_update_valid(update):
    matches_left = np.isin(rules[:, 0], list(update))
    matches_right = np.isin(rules[:, 1], list(update))
    
    both_matches = matches_left & matches_right
    
    # map array values to indices for efficient lookups
    value_to_index = {value: idx for idx, value in enumerate(update)}
    
    for key in set(rules[both_matches][:,0]):
        key_idx = value_to_index.get(key, None)
        
        for value in rules_graph[key]:
            if value_idx := value_to_index.get(value, None):
                if value_idx <= key_idx:
                    return False
    
    return True

In [61]:
sum_valid = 0
for line in updates.splitlines():
    update = np.genfromtxt(line.strip().splitlines(), delimiter=",", dtype=int)
    if is_update_valid(update):
        sum_valid += update[len(update)//2]
sum_valid

np.int64(4609)

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

`75,97,47,61,53` becomes `97,75,47,61,53`.

`61,13,29` becomes `61,29,13`.

`97,13,75,29,47` becomes `97,75,47,29,13`.

After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

In [62]:
def reordered_update(update, rules):
    pass

In [ ]:
sum_corrected = 0
for line in updates.splitlines():
    update = np.genfromtxt(line.strip().splitlines(), delimiter=",", dtype=int)
    if not is_update_valid(update):
        reordered_update = reorder(update)
        sum_corrected += reordered_update[len(update)//2]
        
sum_corrected